# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [43]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [44]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
list_of_files = []

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*events.csv'))
    list_of_files.extend(file_path_list)
    

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [45]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in list_of_files:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('new_event_datafile.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [46]:
# check the number of rows in your csv file
with open('new_event_datafile.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [47]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)


#### Create Keyspace

In [48]:
# Create a Keyspace 
try:
    session.execute ("""Create keyspace if not exists events \
                      with replication = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
                    )
except Excception as e:
    print(e)

#### Set Keyspace

In [49]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('events')
except Exception as e:
    print(e)

### Creating tables to run the following queries. With Apache Cassandra, database tables are modeled on the queries to be run.

### Query 1: Get the artist, song title and song length for a given sessionid and itemsinsession
#### The table for this query is called user_session
#### The partitioning key is sessionid and the clustering column is itemsinsession which enables filtering on these columns. 

In [50]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS user_session"
query = query + "(sessionid int, itemsinsession int, artist text, song text, songlength float, PRIMARY KEY(sessionid,itemsinsession) )"
try:
    session.execute(query);
except Exception as e:
    print(e)
                    

### Load data into user_session table 

In [51]:
# Provide the new file name
file = 'new_event_datafile.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "Insert into user_session (sessionid, itemsinsession,artist,song,songlength)"
        query = query + " values(%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)
 

#### Do a SELECT to verify that the data have been inserted into user_session table

In [52]:
## Add in the SELECT statement to verify the data was entered into the table
query = "select artist,song,songlength from user_session where sessionid=338 and itemsinsession=4"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist,row.song,row.songlength)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2: Get the name of artist, song (sorted by itemInSession) and user (first and last name) for a given userid and sessionid
#### The table supporting this query is called user_songs
#### The composite key is (userid,sessionid) and the clustering column is itemsinsession. The clustering key sorts data by itemsinsession

In [53]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "create table if not exists user_songs"
query = query + "(userid int, sessionid int, itemsinsession int, artist text, song text, user text, Primary key((userid,sessionid),itemsinsession) )"

try:
    session.execute(query);
except Exception as e:
    print(e)

#### Load data into user_songs table

In [54]:
# Load data from the csv file
file = 'new_event_datafile.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "Insert into user_songs (userid, sessionid,itemsinsession, artist, song,user)"
        query = query + " values(%s, %s, %s, %s, %s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1]+" "+line[4]))
        except Exception as e:
            print(e)
        

#### Do a SELECT to verify that the data have been inserted into user_songs table

In [55]:
query = "select artist,song, user from user_songs where userid = 10 and sessionid = 182"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.user)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3: Get every user name (first and last) inthemusic app history who listened to a particular song
#### The table supporting this query is called user_search
#### The primary key is (song,userid) with song being the paritioning key and userid, the clustering column

In [56]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "create table if not exists user_search"
query = query + "(song text, userid int, user text, primary key(song,userid))"

try:
    session.execute(query);
except Exception as e:
    print(e)
                    

#### Load data into user_search table

In [57]:
file = 'new_event_datafile.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "Insert into user_search (song, userid, user)"
        query = query + " values(%s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        try:
            session.execute(query, (line[9],int(line[10]),line[1]+" "+line[4]))
        except Exception as e:
            print(e)
        

#### Do a SELECT to verify that the data have been inserted into user_search table

In [58]:
query = "select user from user_search where song = 'All Hands Against His Own'"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.user)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [59]:
## Drop the table before closing out the sessions
query = "drop table user_session"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_songs"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_search"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [40]:
session.shutdown()
cluster.shutdown()

#### Run the consolidated etl pipeline script etl_cass.py

In [41]:
run -i etl_cass.py

/home/workspace
Total number of files is  30
Reading file  /home/workspace/event_data/2018-11-27-events.csv
Reading file  /home/workspace/event_data/2018-11-04-events.csv
Reading file  /home/workspace/event_data/2018-11-07-events.csv
Reading file  /home/workspace/event_data/2018-11-09-events.csv
Reading file  /home/workspace/event_data/2018-11-19-events.csv
Reading file  /home/workspace/event_data/2018-11-05-events.csv
Reading file  /home/workspace/event_data/2018-11-22-events.csv
Reading file  /home/workspace/event_data/2018-11-16-events.csv
Reading file  /home/workspace/event_data/2018-11-26-events.csv
Reading file  /home/workspace/event_data/2018-11-24-events.csv
Reading file  /home/workspace/event_data/2018-11-29-events.csv
Reading file  /home/workspace/event_data/2018-11-15-events.csv
Reading file  /home/workspace/event_data/2018-11-20-events.csv
Reading file  /home/workspace/event_data/2018-11-06-events.csv
Reading file  /home/workspace/event_data/2018-11-18-events.csv
Reading fi